In [1]:
import pandas as pd
from sklearn.datasets import load_iris

In [4]:
IRIS = load_iris()
iris = pd.DataFrame(data=IRIS.data, columns=IRIS.feature_names)
iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [6]:
iris['class'] = IRIS.target
# iris['class'] = iris['class'].map({0: 'setosa', 1:'versicolor', 2:'virginica'})
iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [7]:
iris['class'] = iris['class'].map({0: 'setosa', 1:'versicolor', 2:'virginica'})
iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


# 데이터 요약
### 1. 그룹화와 집계

In [8]:
iris.groupby(by='class').mean()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
class,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [9]:
iris.groupby(by='class').median()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
class,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3
virginica,6.5,3.0,5.55,2.0


### 2. 도수분포표
- 자료를 몇 개의 구간으로 나누고, 나누어진 각 구간에 속한 자료의 개수를 정리한 표이다. 
- 수치형 데이터가 어떤 모양으로 분포되어 있는지 확인할 수 있다.

In [10]:
pd.Series(iris['class']).value_counts()

class
setosa        50
versicolor    50
virginica     50
Name: count, dtype: int64

In [14]:
iris['petal width level'] = pd.qcut(iris['petal width (cm)'], q=3, labels=['short', 'middle', 'long'])
iris.head()
pd.Series(iris['petal width level']).value_counts().to_frame()

,count
petal width level,
middle,52
short,50
long,48


In [15]:
pd.crosstab(iris['petal width level'], iris['class'])

class,setosa,versicolor,virginica
petal width level,,,
short,50,0,0
middle,0,48,4
long,0,2,46


# 데이터 재구조화
### 1. 피벗 테이블(Pivot Table)
- 원본 데이터의 구조를 바꾸는 것



In [16]:
import pandas as pd
score = {
    '학년': [1, 1, 1, 1, 2, 2],
    '반': ['A', 'A', 'B', 'B', 'C', 'C'],
    '성별': ['여자','남자','여자','남자','여자','남자'],
    '성적': [76, 88, 85, 72, 68, 70]
}

score = pd.DataFrame(score)
score

,학년,반,성별,성적
0,1,A,여자,76
1,1,A,남자,88
2,1,B,여자,85
3,1,B,남자,72
4,2,C,여자,68
5,2,C,남자,70


In [17]:
score = score.pivot_table(
    index=['학년', '반'],
    columns='성별',
    values='성적'
    )
score

성별      남자    여자
학년 반            
1  A  88.0  76.0
   B  72.0  85.0
2  C  70.0  68.0

### 2. 멜트(Melt)
- 피벗 테이블의 반대
- 컬럼으로 나열된 자료를 variable과 value 컬럼으로 압축

In [18]:
score.reset_index().melt(id_vars=['학년', '반'], var_name='성별', value_name='성적')

,학년,반,성별,성적
0,1,A,남자,88.0
1,1,B,남자,72.0
2,2,C,남자,70.0
3,1,A,여자,76.0
4,1,B,여자,85.0
5,2,C,여자,68.0


# 데이터프레임에 함수 적용하기
### 1. apply
- 데이터프레임에 행 또는 열 방향으로 지정한 함수를 수행하고 그 값을 결과로 받는다.
- apply는 집계함수도 적용할 수 있다.

In [19]:
import numpy as np

score.apply(np.sqrt, axis=0)

성별          남자        여자
학년 반                    
1  A  9.380832  8.717798
   B  8.485281  9.219544
2  C  8.366600  8.246211

In [20]:
score.apply(np.max, axis=0)

성별
남자    88.0
여자    85.0
dtype: float64

In [21]:
def plus_five(val):
    return val+5

score.apply(plus_five)

성별      남자    여자
학년 반            
1  A  93.0  81.0
   B  77.0  90.0
2  C  75.0  73.0

In [25]:
from math import ceil
def class_avg(df):
    return ceil((df['남자'] + df['여자'])/2)

score.apply(class_avg, axis=1)


학년  반
1   A    82
    B    79
2   C    69
dtype: int64

### 2. 맵(Map)
- 주어진 각 원소에 함수를 적용한다.
- 함수를 정의하는 lambda와 함께 사용할 수 있다.
- map(lambda)를 사용할 때에는 데이터가 시리즈 형식이어야 한다

In [26]:
score['남자'].map(lambda x: x+5)

학년  반
1   A    93.0
    B    77.0
2   C    75.0
Name: 남자, dtype: float64